In [14]:
from SPARQLWrapper import SPARQLWrapper, JSON
import json
import os

In [15]:
qald9_path = "/home/kardosp/continuethelist/datasets/QALD/9/data/"

In [16]:
with open(os.path.join(qald9_path, "qald-9-train-multilingual.json"), "r") as f:
    train_json_data = json.load(f)

with open(os.path.join(qald9_path, "qald-9-test-multilingual.json"), "r") as f:
    test_json_data = json.load(f)

In [44]:
# sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql = SPARQLWrapper("http://localhost:8890/sparql")
# "http://localhost:8890/sparql"
# "http://dbpedia.org/sparql"
sparql.setReturnFormat(JSON)

def run_sparql_query(query):
    sparql.setQuery(query)

    results = None
    try:
        ret = sparql.queryAndConvert()

        for r in ret["results"]["bindings"]:
            if results is None:
                results = [r]
            else:
                results.append(r)
    except Exception as e:
        print("Error -->", e)
    return results

def select_question(q, lang="en"):
    for item in q:
        if item["language"] == lang:
            return item["string"]
    return None

def add_answer_to_questions(input_df):

    questions = list()
    for i in range(len(input_df)):
        print(i)
        result = run_sparql_query(input_df[i]["query"]["sparql"])

        result_extracted = list()
        if result is not None:
            for item in result:
                if "callret-0" in item:
                    result_extracted.append(item["callret-0"]["value"])
                elif "uri" in item:
                    result_extracted.append(item["uri"]["value"])
                else:
                    try:
                        result_extracted.append(item[list(item.keys())[0]]["value"])
                    except Exception as e:
                        print("New enemy encountered: ", item, e)
                # elif "carbs" in item:
                #     result_extracted.append(item["carbs"]["value"])
                # elif "date" in item:
                #     result_extracted.append(item["date"]["value"])
                # elif "c" in item:
                #     result_extracted.append(item["c"]["value"])
                # elif "ni" in item:
                #     result_extracted.append(item["ni"]["value"])
                # elif "num" in item:
                #     result_extracted.append(item["num"]["value"])
                # elif "n" in item:
                #     result_extracted.append(item["n"]["value"])
                # elif "string" in item:
                #     result_extracted.append(item["string"]["value"])
                # elif "x" in item:
                #     result_extracted.append(item["x"]["value"])
                # elif "books" in item:
                #     result_extracted.append(item["books"]["value"])
                # else:
                #     print("New enemy encountered: ", item)

        result_container = dict()
        result_container["result"] = result_extracted
        result_container["_id"] = input_df[i]["id"]
        result_container["corrected_question"] = select_question(input_df[i]["question"])
        result_container["sparql_query"] = input_df[i]["query"]
        result_container["expected_query_result"] = [item["uri"]["value"] for item in train_json_data["questions"][0]["answers"][0]["results"]["bindings"]]

        questions.append(result_container)

    return questions

In [31]:
add_answer_to_questions([train_json_data["questions"][0]])

0


[{'result': ['http://dbpedia.org/resource/Chandragupta_(board_game)',
   "http://dbpedia.org/resource/Washington's_War",
   'http://dbpedia.org/resource/Twilight_Struggle',
   'http://dbpedia.org/resource/Paths_of_Glory_(board_game)',
   'http://dbpedia.org/resource/Commands_&_Colors:_Ancients',
   'http://dbpedia.org/resource/Fields_of_Fire_(game)',
   'http://dbpedia.org/resource/Sword_of_Rome'],
  '_id': '1',
  'corrected_question': 'List all boardgames by GMT.',
  'sparql_query': {'sparql': 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT ?uri WHERE { ?uri dbo:publisher res:GMT_Games }'},
  'expected_query_result': ["http://dbpedia.org/resource/Washington's_War",
   'http://dbpedia.org/resource/Fields_of_Fire_(game)',
   'http://dbpedia.org/resource/Commands_&_Colors:_Ancients',
   'http://dbpedia.org/resource/A_World_At_War',
   'http://dbpedia.org/resource/Sword_of_Rome',
   'http://

In [45]:
questions_train = add_answer_to_questions(train_json_data["questions"])
questions_test = add_answer_to_questions(test_json_data["questions"])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
Error --> 'results'
104
105
106
Error --> 'results'
107
108
109
110
111
112
113
114
Error --> QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 3: Undefined namespace prefix at 'dbo' before '?x'\n\nSPARQL query:\ndefine sql:big-data-const 0\n#output-format:application/sparql-results+json\nSELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Mount_McKinley> dbo:wikiPageRedirects ?x . ?x <http://dbpedia.org/ontology/locatedInArea> ?uri. ?uri rdf:type yago:WikicatStatesOfTheUnitedStates }"
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
Error --> 'resu

In [46]:
with open(os.path.join(qald9_path, "qald9_dump_w_answers_train.json"), "w") as f:
    json.dump(questions_train, f)

with open(os.path.join(qald9_path, "qald9_dump_w_answers_test.json"), "w") as f:
    json.dump(questions_test, f)

# Filter questions

In [47]:
with open(os.path.join(qald9_path, "qald9_dump_w_answers_train.json"), "r") as f:
    questions_train = json.load(f)

with open(os.path.join(qald9_path, "qald9_dump_w_answers_test.json"), "r") as f:
    questions_test = json.load(f)

In [48]:
valid_questions_train = list()
valid_questions_test = list()

lower_bound = 10
upper_bound = 100

for item in questions_train:
    if upper_bound > len(item["result"]) > lower_bound:
        valid_questions_train.append(item)

for item in questions_test:
    if upper_bound > len(item["result"]) > lower_bound:
        valid_questions_test.append(item)

In [49]:
print(len(valid_questions_train), len(valid_questions_test))

54 21


In [50]:
with open(os.path.join(qald9_path, "qald9_dump_valid_train.json"), "w") as f:
    json.dump(valid_questions_train, f)

with open(os.path.join(qald9_path, "qald9_dump_valid_test.json"), "w") as f:
    json.dump(valid_questions_test, f)

In [51]:
from collections import Counter
print(Counter([len(item["result"]) for item in questions_train]))

Counter({1: 206, 0: 43, 2: 27, 3: 15, 7: 10, 4: 8, 5: 7, 8: 7, 13: 5, 10: 5, 11: 5, 12: 4, 15: 4, 9: 3, 6: 3, 34: 2, 128: 2, 55: 2, 45: 2, 56: 2, 49: 2, 22: 2, 24: 2, 2097: 2, 78: 1, 562: 1, 4064: 1, 26: 1, 54: 1, 117: 1, 17: 1, 111: 1, 27: 1, 42: 1, 151: 1, 107: 1, 85: 1, 2232: 1, 44: 1, 33: 1, 171: 1, 1117: 1, 220: 1, 59: 1, 63: 1, 47: 1, 18: 1, 683: 1, 144: 1, 35: 1, 23: 1, 122: 1, 19: 1, 360: 1, 527: 1, 38: 1, 68: 1, 25: 1, 872: 1, 39: 1, 32: 1, 16: 1})
